In [0]:
from __future__ import absolute_import,division,print_function,unicode_literals
try:
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

In [0]:
from tensorflow.keras.layers import Dense,Conv2D,Flatten
from tensorflow.keras import Model

In [0]:
mnist=tf.keras.datasets.mnist
(x_train,y_train),(x_test,y_test)=mnist.load_data()
x_train , x_test=x_train/255.0 , x_test/255.0


x_train=x_train[...,tf.newaxis]
x_test=x_test[...,tf.newaxis]


In [0]:
train_ds=tf.data.Dataset.from_tensor_slices((x_train,y_train)).shuffle(10000).batch(32)
test_ds= tf.data.Dataset.from_tensor_slices((x_test,y_test)).batch(32)

In [0]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10, activation='softmax')

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

# Create an instance of the model
model = MyModel()

In [0]:
loss_object=tf.keras.losses.SparseCategoricalCrossentropy()
optimizer=tf.keras.optimizers.Adam()

In [0]:
train_loss=tf.keras.metrics.Mean(name='train_loss')
train_accuracy=tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss=tf.keras.metrics.Mean(name='test_loss')
test_accuracy=tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [0]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

In [0]:
@tf.function
def test_step(image,labels):
  predictions=model(image)
  t_loss=loss_object(labels,predictions)

  test_loss(t_loss)
  test_accuracy(labels,predictions)

In [0]:
EPOCHS = 5

for epoch in range(EPOCHS):
  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print(template.format(epoch+1,
                        train_loss.result(),
                        train_accuracy.result()*100,
                        test_loss.result(),
                        test_accuracy.result()*100))

  # Reset the metrics for the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()



 

Epoch 1, Loss: 0.11908502876758575, Accuracy: 96.43859100341797, Test Loss: 0.05384168401360512, Test Accuracy: 98.25999450683594
Epoch 2, Loss: 0.03660295158624649, Accuracy: 98.85832977294922, Test Loss: 0.051945384591817856, Test Accuracy: 98.36000061035156
Epoch 3, Loss: 0.01864124834537506, Accuracy: 99.39167022705078, Test Loss: 0.06484683603048325, Test Accuracy: 98.00999450683594
Epoch 4, Loss: 0.012575095519423485, Accuracy: 99.57333374023438, Test Loss: 0.07017344981431961, Test Accuracy: 98.05999755859375
Epoch 5, Loss: 0.009402050636708736, Accuracy: 99.68333435058594, Test Loss: 0.07736282050609589, Test Accuracy: 98.2699966430664
